In [1]:
from google.colab import drive
drive.mount('/content/drive')

GDRIVE_PROJECT = "/content/drive/MyDrive/lyric-generation-ai"
COLAB_PROJECT = "/content/lyric-generation-ai"

!rm -rf {COLAB_PROJECT}
!git clone https://github.com/risheetnair/lyric-generation-ai.git {COLAB_PROJECT}
%cd {COLAB_PROJECT}

!pip install -r requirements.txt

import os

for folder in ["data/raw", "data/processed", "results", "models"]:
    os.makedirs(f"{GDRIVE_PROJECT}/{folder}", exist_ok=True)
    if os.path.exists(folder):
        !rm -rf {folder}
    !ln -s {GDRIVE_PROJECT}/{folder} {folder}

print("✅ Setup complete. Repo cloned, Drive mounted, persistent folders linked.")
!ls -R .


Cloning into 'lyric-generation-ai'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 12), reused 19 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 5.83 KiB | 5.83 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/lyric-generation-ai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
.:
data  models  notebooks  README.md  requirements.txt  results  src  writeup

./data:
processed  raw

./data/processed:
README.md

./data/raw:
README.md

./models:
README.md

./notebooks:
01_data_prep.ipynb	03_genre_conditioning.ipynb  README.md
02_baseline_gpt2.ipynb	04_evaluation.ipynb

./results:
metrics  samples

./results/metrics:
README.md

./results/samples:
README.md

./src:
README.md

./writeup:
README.md


In [12]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

RAW = Path("data/raw")
PROC = Path("data/processed")
PROC.mkdir(parents=True, exist_ok=True)

print("RAW ->", RAW.resolve())
print("PROC ->", PROC.resolve())

RAW -> /content/drive/MyDrive/lyric-generation-ai/data/raw
PROC -> /content/drive/MyDrive/lyric-generation-ai/data/processed


In [13]:
# Try to find a CSV in data/raw
csv_candidates = list(RAW.glob("*.csv"))
if not csv_candidates:
    raise FileNotFoundError("No CSV found in data/raw.")
for i, p in enumerate(csv_candidates, 1):
    print(f"{i}. {p.name}")

csv_path = csv_candidates[0]
print("\nUsing:", csv_path.name)

df = pd.read_csv(csv_path)
print("Raw shape:", df.shape)
df.head()


1. artists-data.csv
2. lyrics-data.csv

Using: artists-data.csv
Raw shape: (4168, 5)


,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
